# NER + parquet/duckdbEvaluation 

In [1]:
import spacy
import mlflow
import mlflow.spacy
import polars as pl
import scipy
import duckdb
import numpy as np
import json
import timeit
import matplotlib.pyplot as plt
from rank_bm25 import BM25Okapi

In [2]:
import os
import sys
from dotenv import load_dotenv
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))
sys.path.insert(0, os.path.abspath('../../..'))
from app.evaluation import reciprocal_rank, rank_BOW_results, precision_at_k
from app.data_handling.NER_parquet import (
process_ingredient,
transform_ingredients_to_tokens,
transform_data_to_tokens,
construct_ingredient_query,
ParquetDefinition,
create_parquet_file
)
load_dotenv()

True

# Set up definitions
Datasets:
 - 1k_processed
 - 10k
 - 100k
 - full

In [3]:
dataset_name = '1k_processed'
run_name = 'line_only_pre_lemma_v5'
model_name = run_name
force_overwrite_pq = True
model_uri = 'models:/recipe_NER@prod'
#model_uri = 'runs:/e00edb7abff845f3913810918de00fe8/model'

In [4]:
with open("../../../data/eval_data/processed_data.json", "r") as f:
    query_ds = pl.read_json(f)
with open("../../../data/eval_data/human_annotations.json", "r") as f:
    human_query_ds = pl.read_json(f)

In [5]:
if dataset_name == '1k_processed':
    eval_ds = query_ds
elif dataset_name == '10k':
    with open("../../../data/eval_data/eval_10k.json", "r") as f:
        eval_ds = pl.read_json(f)
elif dataset_name == '100k':
    with open("../../../data/eval_data/eval_100k.json", "r") as f:
        eval_ds = pl.read_json(f)

In [6]:
results_df = pl.DataFrame(
    schema={
        'Model': pl.String,
        'Eval_Task': pl.String,
        "Dataset": pl.String,
        "MRR": pl.Float64,
        "P@3": pl.Float64,
        "Time_Tot": pl.Float64,
        "Time_Query": pl.Float64,
        "Error_Count": pl.Int32
    })
results_df

Model,Eval_Task,Dataset,MRR,P@3,Time_Tot,Time_Query,Error_Count
str,str,str,f64,f64,f64,f64,i32


## Running code

In [7]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL"))

In [8]:
#model_uri = 'models:/recipe_NER@prod'
NER_model = mlflow.spacy.load_model(model_uri)

In [9]:
pq_filen = f"{dataset_name}.parquet"

In [10]:
def rank_results(df: pl.DataFrame, query: list[str]):
    if df.shape[0] == 0:
        return df
    corpus = list(df['tokens'])
    bm25 = BM25Okapi(corpus)
    scores = bm25.get_scores(query)
    ranked_ds = df.with_columns(
        pl.Series(name='rank', values=scores)
    ).sort('rank', descending=True)
    return ranked_ds

In [11]:
start = timeit.default_timer()
tokens, preps, optionals, varieties, brands, alt_foods = transform_data_to_tokens(
    eval_ds['ingredients'], NER_model,
    True, True, True
)

In [12]:
inp = [
    ParquetDefinition(data=tokens, name="tokens"),
    ParquetDefinition(data=preps, name="preps", is_map=True),
    ParquetDefinition(data=optionals, name="optionals"),
    ParquetDefinition(data=varieties, name="varieties", is_map=True),
    ParquetDefinition(data=brands, name="brands", is_map=True),
    ParquetDefinition(data=alt_foods, name="alt_foods"),
]
create_parquet_file(pq_filen, eval_ds.select(pl.exclude('tfidf')), inp, force_overwrite=force_overwrite_pq)
time = timeit.default_timer() - start

In [13]:
print(f"Dataset transformation took {time}s, or {time/eval_ds.shape[0]}s/row")

Dataset transformation took 59.46693890495226s, or 0.059466938904952256s/row


In [14]:
def evaluation_task_NER(parquet_path:str, query_ds: pl.DataFrame, query_field: str, results_df: pl.DataFrame, task_name: str, debug = False, model_name='ner'):
    tot_rr = 0
    tot_prec_at_3 = 0
    tot_evals = 0
    errors = 0
    start = timeit.default_timer()
    for index, query in query_ds.select('index', query_field).iter_rows():
        if query_field == 'google_search_query_processed':
            query_obj = transform_ingredients_to_tokens(query['ingredients'], NER_model)
        else:
            query_obj = transform_ingredients_to_tokens(query, NER_model)
        sql = construct_ingredient_query(parquet_path, query_obj['foods'], query_obj['preps'])
        try:
            results = duckdb.sql(sql).pl()
            if debug and results.shape[0] == 0:
                print(sql)
                print("-- Zero results --")
                print(index)
                print("- Target -")
                t = duckdb.sql(f"SELECT preps, tokens FROM '{parquet_path}' WHERE index == {index}").pl()
                print(list(t[0, 1]))
                print(list(t[0, 0]))
                #print(" ".join(list(t[0])))
                print("- Query -")
                print(" ".join(query_obj['foods']))
                print(query_obj['preps'])
            results = rank_results(results, query_obj['foods'])
            tot_rr += reciprocal_rank(results, index)
            tot_prec_at_3 += precision_at_k(results, index)
            tot_evals += 1
        except Exception as e:
            errors += 1
            print(e)
            continue
    time = timeit.default_timer() - start
    mrr = tot_rr/tot_evals
    mprec3 = tot_prec_at_3 / tot_evals
    result_data = {
        "Model": [model_name],
        'Eval_Task': [task_name],
        'Dataset': [dataset_name],
        "MRR": [mrr],
        "P@3": [mprec3],
        "Time_Tot": time,
        "Time_Query": time/tot_evals,
        "Error_Count": errors
    }
    result_df = pl.DataFrame(result_data)
    results_df = results_df.vstack(result_df)
    return results_df

In [15]:
results_df = evaluation_task_NER(pq_filen, query_ds, 'ingredients', results_df, 'identity', model_name=model_name)
results_df = evaluation_task_NER(pq_filen, query_ds, 'main_food_items_and_preparations', results_df, 'llm1', model_name=model_name)
results_df = evaluation_task_NER(pq_filen, query_ds, 'important_ingredients_for_search', results_df, 'llm2', model_name=model_name)
results_df = evaluation_task_NER(pq_filen, query_ds, 'google_search_query_processed', results_df, 'llm3', model_name=model_name)

Binder Error: Referenced column "duncan_hines_moist_deluxe_devil_'s_food" not found in FROM clause!
Candidate bindings: "varieties", "main_food_items_and_preparations", "important_ingredients_for_search", "ingredients", "google_search_query_processed"
Binder Error: Referenced column "'s" not found in FROM clause!
Candidate bindings: "google_search_query", "google_search_query_processed", "main_food_items_and_preparations"
Binder Error: Referenced column "d'espelette" not found in FROM clause!
Candidate bindings: "index", "ingredients", "preps", "tokens", "brands"
Binder Error: Referenced column "campbell_'s_condense_tomato_soup" not found in FROM clause!
Candidate bindings: "main_food_items_and_preparations", "important_ingredients_for_search", "google_search_query_processed", "preps", "alt_foods"
Binder Error: Referenced column "bottlefrank_'s_original" not found in FROM clause!
Candidate bindings: "brands", "title", "tokens", "important_ingredients_for_search", "optionals"
Binder Err

In [16]:
results_df = evaluation_task_NER(pq_filen, human_query_ds, 'human_ingredients', results_df, 'gold_ingredients', model_name=model_name)
results_df = evaluation_task_NER(pq_filen, human_query_ds, 'human_key_ingredients', results_df, 'gold_key_ingredients', model_name=model_name)

In [17]:
results_df

Model,Eval_Task,Dataset,MRR,P@3,Time_Tot,Time_Query,Error_Count
str,str,str,f64,f64,f64,f64,i32
"""line_only_pre_lemma_v5""","""identity""","""1k_processed""",0.989782,0.99799,70.821171,0.071177,5
"""line_only_pre_lemma_v5""","""llm1""","""1k_processed""",0.569989,0.580581,60.77093,0.060832,1
"""line_only_pre_lemma_v5""","""llm2""","""1k_processed""",0.443578,0.478,32.486376,0.032486,0
"""line_only_pre_lemma_v5""","""llm3""","""1k_processed""",0.262138,0.297,27.698316,0.027698,0
"""line_only_pre_lemma_v5""","""gold_ingredients""","""1k_processed""",0.32,0.32,2.093862,0.041877,0
"""line_only_pre_lemma_v5""","""gold_key_ingredients""","""1k_processed""",0.542953,0.54,1.342176,0.026844,0


In [18]:
from datetime import datetime
date_time = datetime.now().strftime("%Y_%m_%d__%H_%M")

In [19]:
date_time

'2025_06_11__23_09'

In [20]:
results_df.write_parquet(f"../../../data/results/NER/{run_name}_{date_time}.parquet")

In [21]:
pl.read_parquet(f"../../../data/results/NER/{run_name}_{date_time}.parquet")

Model,Eval_Task,Dataset,MRR,P@3,Time_Tot,Time_Query,Error_Count
str,str,str,f64,f64,f64,f64,i32
"""line_only_pre_lemma_v5""","""identity""","""1k_processed""",0.989782,0.99799,70.821171,0.071177,5
"""line_only_pre_lemma_v5""","""llm1""","""1k_processed""",0.569989,0.580581,60.77093,0.060832,1
"""line_only_pre_lemma_v5""","""llm2""","""1k_processed""",0.443578,0.478,32.486376,0.032486,0
"""line_only_pre_lemma_v5""","""llm3""","""1k_processed""",0.262138,0.297,27.698316,0.027698,0
"""line_only_pre_lemma_v5""","""gold_ingredients""","""1k_processed""",0.32,0.32,2.093862,0.041877,0
"""line_only_pre_lemma_v5""","""gold_key_ingredients""","""1k_processed""",0.542953,0.54,1.342176,0.026844,0
